# Step 1 — Data Preparation (SHP Main Study) — Waves 2019–2023 (W21–W25)

This notebook builds an **analysis-ready long panel dataset** for **2019–2023** from:

- Individual (person) files (`*_p_user.dta`)
- Household files (`*_h_user.dta`)
- **Imputed personal income** (`imputed_income_pers_long_shp.dta`)

It will:
1. Load each wave folder (`data/raw/W21_2019` … `W25_2023`)
2. Merge person + household for each year
3. Harmonize key variables across waves using **robust heuristics** (SHP names differ by wave)
4. Merge imputed income (person-level) by `idpers` + `year` (or closest matching key)
5. Export: `analysis_dataset_step1_panel_2019_2023.csv`

> If you prefer a pooled cross-section, you can still use the output and treat it as pooled data.


In [6]:
%pip install --upgrade pip
%pip install pandas numpy pyreadstat


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import re
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

# -------------------------------------------------------------------
# CONFIG (auto-detect SHP folders inside data/raw)
# -------------------------------------------------------------------
PROJECT_ROOT = os.getcwd()
RAW_ROOT = os.path.join(PROJECT_ROOT, "data", "raw")

# Main wave folder (contains W21_2019 ... W25_2023)
WAVES_ROOT = None
if os.path.isdir(RAW_ROOT):
    # common exact folder name
    p = os.path.join(RAW_ROOT, "SHP-Data-W1-W25-STATA")
    if os.path.isdir(p):
        WAVES_ROOT = p
    else:
        # fallback: search one level deep
        for entry in os.listdir(RAW_ROOT):
            cand = os.path.join(RAW_ROOT, entry)
            if os.path.isdir(cand) and re.search(r"W1[-_]?W25", entry, flags=re.IGNORECASE):
                WAVES_ROOT = cand
                break

# Income file (imputed income)
INCOME_FILE = None
if os.path.isdir(RAW_ROOT):
    for root, dirs, files in os.walk(RAW_ROOT):
        for f in files:
            if f.lower() == "imputed_income_pers_long_shp.dta":
                INCOME_FILE = os.path.join(root, f)
                break
        if INCOME_FILE:
            break

WAVE_FOLDERS = ["W21_2019", "W22_2020", "W23_2021", "W24_2022", "W25_2023"]

print("Working directory:", PROJECT_ROOT)
print("RAW_ROOT exists:", os.path.isdir(RAW_ROOT), "->", RAW_ROOT)
print("WAVES_ROOT detected:", WAVES_ROOT)
print("Income file detected:", INCOME_FILE)

if WAVES_ROOT:
    print("Wave folders present:", [wf for wf in WAVE_FOLDERS if os.path.isdir(os.path.join(WAVES_ROOT, wf))])
else:
    print("Wave folders present: []")


Working directory: /workspaces/EM_Project
RAW_ROOT exists: True -> /workspaces/EM_Project/data/raw
WAVES_ROOT detected: /workspaces/EM_Project/data/raw/SHP-Data-W1-W25-STATA
Income file detected: /workspaces/EM_Project/data/raw/SHP-Data-Imputed-Income-Wealth-STATA/imputed_income_pers_long_shp.dta
Wave folders present: ['W21_2019', 'W22_2020', 'W23_2021', 'W24_2022', 'W25_2023']


## Helper functions

In [8]:
def find_file(folder: str, pattern: str) -> str:
    for f in os.listdir(folder):
        if re.search(pattern, f, flags=re.IGNORECASE):
            return os.path.join(folder, f)
    raise FileNotFoundError(f"No file matching {pattern} found in {folder}")

def safe_to_numeric(s):
    return pd.to_numeric(s, errors="coerce")

def pick_first_existing(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

def pick_by_regex(df, patterns):
    cols = list(df.columns)
    for pat in patterns:
        rx = re.compile(pat, flags=re.IGNORECASE)
        hits = [c for c in cols if rx.search(c)]
        if hits:
            return hits[0]
    return None

def wave_num_from_folder(folder_name: str) -> int:
    m = re.match(r"W(\d{1,2})_", folder_name)
    return int(m.group(1)) if m else None

def year_from_folder(folder_name: str) -> int:
    m = re.match(r"W\d{1,2}_(\d{4})", folder_name)
    return int(m.group(1)) if m else None


## 2) Load & merge each wave (P + H)

In [9]:
panel_parts = []

for wf in WAVE_FOLDERS:
    folder = os.path.join(WAVES_ROOT, wf)
    if not os.path.isdir(folder):
        raise FileNotFoundError(f"Wave folder not found: {folder}. Check WAVES_ROOT and that W21_2019..W25_2023 exist.")

    wave = wave_num_from_folder(wf)
    year = year_from_folder(wf)

    p_path = find_file(folder, r"_p_.*user.*\.dta$")
    h_path = find_file(folder, r"_h_.*user.*\.dta$")

    print("\n" + "="*90)
    print(f"Loading {wf} | wave={wave}, year={year}")
    print("P:", os.path.basename(p_path))
    print("H:", os.path.basename(h_path))

    df_p = pd.read_stata(p_path, convert_categoricals=False)
    df_h = pd.read_stata(h_path, convert_categoricals=False)

    if "idpers" not in df_p.columns:
        raise KeyError("idpers not found in person file")

    # Try household id variants
    idhous_p = pick_first_existing(df_p, ["idhous", f"idhous{wave:02d}", f"idhous{wave}", f"idhous{str(year)[-2:]}"])
    idhous_h = pick_first_existing(df_h, ["idhous", f"idhous{wave:02d}", f"idhous{wave}", f"idhous{str(year)[-2:]}"])
    if idhous_p is None:
        idhous_p = pick_by_regex(df_p, [r"^idhous"])
    if idhous_h is None:
        idhous_h = pick_by_regex(df_h, [r"^idhous"])
    if idhous_p is None or idhous_h is None:
        raise KeyError("No household id column found for merge (idhous*)")

    df = df_p.merge(df_h, left_on=idhous_p, right_on=idhous_h, how="left", suffixes=("", "_h"))
    df["year"] = year
    df["wave"] = wave
    df["idpers"] = df["idpers"]
    df["idhous"] = df[idhous_p]

    panel_parts.append(df)

df_long = pd.concat(panel_parts, ignore_index=True)
print("\nLong panel shape:", df_long.shape)
display(df_long[["idpers","idhous","year","wave"]].head())



Loading W21_2019 | wave=21, year=2019
P: shp19_p_user.dta
H: shp19_h_user.dta

Loading W22_2020 | wave=22, year=2020
P: shp20_p_user.dta
H: shp20_h_user.dta

Loading W23_2021 | wave=23, year=2021
P: shp21_p_user.dta
H: shp21_h_user.dta

Loading W24_2022 | wave=24, year=2022
P: shp22_p_user.dta
H: shp22_h_user.dta

Loading W25_2023 | wave=25, year=2023
P: shp23_p_user.dta
H: shp23_h_user.dta

Long panel shape: (90910, 3420)


,idpers,idhous,year,wave
0,5101,51,2019,21
1,5103,52,2019,21
2,5104,51,2019,21
3,5201,52,2019,21
4,13101,131,2019,21


## 3) Harmonize core variables (best-effort)

In [10]:

import time

t0 = time.time()

# NUR benötigte Spalten ziehen (schnell + RAM-schonend)
need = [
    "idpers","idhous","year","wave",
    "age19","age20","age21","age22","age23",
    "sex19","sex20","sex21","sex22","sex23",
    "edyear19","edyear20","edyear21","edyear22","edyear23",
    "isced19","isced20","isced21","isced22","isced23",
    "p19a04","p20a04","p21a04","p22a04","p23a04",
    "p19c01","p20c01","p21c01","p22c01","p23c01",
]
need = [c for c in need if c in df_long.columns]

df_tmp = df_long[need].copy()

# Sicherstellen: wave ist 1D numeric
wave = pd.to_numeric(df_tmp["wave"], errors="coerce").to_numpy()

df_long2 = df_tmp[["idpers","idhous","year","wave"]].copy()

# Helper: pick the right column by wave using np.where cascade
def by_wave(col19, col20, col21, col22, col23):
    arr = np.full(len(df_tmp), np.nan, dtype="float64")
    if col19 in df_tmp: arr = np.where(wave == 21, pd.to_numeric(df_tmp[col19], errors="coerce"), arr)
    if col20 in df_tmp: arr = np.where(wave == 22, pd.to_numeric(df_tmp[col20], errors="coerce"), arr)
    if col21 in df_tmp: arr = np.where(wave == 23, pd.to_numeric(df_tmp[col21], errors="coerce"), arr)
    if col22 in df_tmp: arr = np.where(wave == 24, pd.to_numeric(df_tmp[col22], errors="coerce"), arr)
    if col23 in df_tmp: arr = np.where(wave == 25, pd.to_numeric(df_tmp[col23], errors="coerce"), arr)
    return arr

df_long2["age"]        = by_wave("age19","age20","age21","age22","age23")
df_long2["sex"]        = by_wave("sex19","sex20","sex21","sex22","sex23")
df_long2["edyear"]     = by_wave("edyear19","edyear20","edyear21","edyear22","edyear23")
df_long2["isced"]      = by_wave("isced19","isced20","isced21","isced22","isced23")
df_long2["sport_raw"]  = by_wave("p19a04","p20a04","p21a04","p22a04","p23a04")
df_long2["health_raw"] = by_wave("p19c01","p20c01","p21c01","p22c01","p23c01")

print("✅ FAST harmonization took", round(time.time() - t0, 1), "seconds")

print("Missing share:")
print(df_long2[["age","sex","edyear","isced","sport_raw","health_raw"]].isna().mean().sort_values())

df_long2.head(10)


✅ FAST harmonization took 0.0 seconds
Missing share:
age           0.0
sex           0.0
edyear        0.0
isced         0.0
sport_raw     0.0
health_raw    0.0
dtype: float64


,idpers,idhous,year,wave,age,sex,edyear,isced,sport_raw,health_raw
0,5101,51,2019,21,58.0,1.0,19.0,51.0,2.0,2.0
1,5103,52,2019,21,27.0,1.0,9.0,20.0,-3.0,-3.0
2,5104,51,2019,21,58.0,2.0,12.0,32.0,-3.0,-3.0
3,5201,52,2019,21,25.0,2.0,12.0,32.0,-3.0,-3.0
4,13101,131,2019,21,47.0,1.0,12.0,32.0,-3.0,-3.0
5,13102,131,2019,21,46.0,2.0,12.0,32.0,2.0,2.0
6,13103,131,2019,21,11.0,2.0,5.0,10.0,-3.0,-3.0
7,13104,131,2019,21,9.0,1.0,2.0,10.0,-3.0,-3.0
8,21101,211,2019,21,60.0,2.0,12.0,32.0,7.0,2.0
9,27101,271,2019,21,51.0,2.0,9.0,20.0,3.0,3.0


In [14]:
import numpy as np

# typische SHP missing codes (kann je nach Modul leicht variieren)
missing_codes = [-1, -2, -3, -4, -5, -6, -7, -8, -9]

cols_to_clean = ["sport_raw", "health_raw", "age", "sex", "edyear", "isced"]
for c in cols_to_clean:
    df_long2[c] = df_long2[c].replace(missing_codes, np.nan)

print("Missing share after recode:")
print(df_long2[cols_to_clean].isna().mean().sort_values())


Missing share after recode:
sex           0.000044
age           0.000737
isced         0.012111
edyear        0.012683
health_raw    0.342548
sport_raw     0.488186
dtype: float64


## 4) Merge imputed personal income (2019–2023)

In [15]:
import time
import re
import numpy as np
import pandas as pd

t0_all = time.time()

if INCOME_FILE is None:
    raise FileNotFoundError("imputed_income_pers_long_shp.dta not found under data/raw")

# Load income file
t0 = time.time()
income = pd.read_stata(INCOME_FILE, convert_categoricals=False)
print("Income file shape:", income.shape, "| read_stata took", round(time.time() - t0, 1), "s")

if "idpers" not in income.columns:
    raise KeyError("Income file does not contain 'idpers'.")

# Detect time key
time_key = None
for cand in ["year", "yr", "syear", "survey_year", "wave"]:
    if cand in income.columns:
        time_key = cand
        break
if time_key is None:
    time_key = pick_by_regex(income, [r"^year$", r"^wave$", r".*year.*"])

print("Detected income time key:", time_key)
if time_key is None:
    raise KeyError("Could not detect a time key in income file.")

# Build year series (without copying entire df first)
if str(time_key).lower() == "wave":
    wave_to_year = {21: 2019, 22: 2020, 23: 2021, 24: 2022, 25: 2023}
    year_series = pd.to_numeric(income[time_key], errors="coerce").map(wave_to_year)
else:
    year_series = pd.to_numeric(income[time_key], errors="coerce")

mask = year_series.between(2019, 2023, inclusive="both")

t0 = time.time()
income2 = income.loc[mask, :].copy()
income2["year"] = year_series.loc[mask].values
print("Income filtered shape (2019–2023):", income2.shape, "| filter+copy took", round(time.time() - t0, 1), "s")

# ------------------------------------------------------------
# Choose correct income variable (labels first)
# ------------------------------------------------------------
income_var = None
try:
    import pyreadstat
    _, meta = pyreadstat.read_dta(INCOME_FILE)

    keywords = ["income", "earn", "wage", "salary", "labour", "labor", "gross", "net", "disposable"]

    label_hits = []
    for name, label in zip(meta.column_names, meta.column_labels):
        lab = (label or "").lower()
        if any(k in lab for k in keywords):
            label_hits.append(name)

    exclude = re.compile(r"^(id|idh|filter|wgt|weight|year|wave)", re.IGNORECASE)
    label_hits = [c for c in label_hits if c in income2.columns]
    label_hits = [c for c in label_hits if (c in income2.select_dtypes(include=[np.number]).columns) and not exclude.search(c)]

    if label_hits:
        income_var = label_hits[0]

    print("Income label hits (usable):", label_hits[:10], "… total:", len(label_hits))
except Exception as e:
    print("pyreadstat label search skipped/failed:", repr(e))

# Fallback if labels don't work
if income_var is None:
    num_cols = income2.select_dtypes(include=[np.number]).columns.tolist()
    exclude = re.compile(r"^(id|idh|filter|wgt|weight|year|wave)", re.IGNORECASE)
    candidates = [c for c in num_cols if not exclude.search(c)]

    name_like = [c for c in candidates if re.search(r"inc|income|earn|wage|salary|gross|net|hhinc", c, flags=re.IGNORECASE)]
    if name_like:
        income_var = name_like[0]
    else:
        best, best_score = None, -1
        for c in candidates:
            s = pd.to_numeric(income2[c], errors="coerce")
            nonmiss = s.notna().mean()
            if nonmiss < 0.20:
                continue
            std = s.std(skipna=True)
            pos_share = (s.dropna() > 0).mean() if s.notna().any() else 0
            uniq = s.nunique(dropna=True)
            if uniq <= 5 and std < 5:
                continue
            score = nonmiss * 1.0 + np.log1p(std) * 0.8 + pos_share * 0.5 + (np.log1p(uniq) / 10)
            if score > best_score:
                best_score, best = score, c
        income_var = best

print("Chosen income variable:", income_var)
if income_var is None:
    raise ValueError("Could not select an income variable. Please inspect income2 columns/labels.")

# Keep only needed columns for merge
income_keep = income2[["idpers", "year", income_var]].copy()
income_keep = income_keep.rename(columns={income_var: "income_imputed"})

# ------------------------------------------------------------
# IMPORTANT SPEED FIX: reduce df_long2 before merge (3420 cols -> ~10 cols)
# ------------------------------------------------------------
t0 = time.time()
core_cols = ["idpers", "idhous", "year", "wave", "age", "sex", "edyear", "isced", "sport_raw", "health_raw"]
core_cols = [c for c in core_cols if c in df_long2.columns]
df_core = df_long2[core_cols].copy()
print("df_core shape:", df_core.shape, "| core extract took", round(time.time() - t0, 1), "s")

# Merge to panel (fast now)
t0 = time.time()
df_panel = df_core.merge(income_keep, on=["idpers", "year"], how="left")
print("After income merge:", df_panel.shape, "| merge took", round(time.time() - t0, 1), "s")

print("Head (income):")
print(df_panel[["idpers", "year", "income_imputed"]].head(10))

print("TOTAL income block took", round(time.time() - t0_all, 1), "s")


Income file shape: (216412, 55) | read_stata took 0.1 s
Detected income time key: year
Income filtered shape (2019–2023): (59911, 55) | filter+copy took 0.0 s
Income label hits (usable): ['iptotni', 'iempyni', 'iindyni', 'ioasiyi', 'ipenyi', 'iuneyi', 'iwelyi', 'iinsyi', 'ipihyi', 'ipnhyi'] … total: 36
Chosen income variable: iptotni
df_core shape: (90910, 10) | core extract took 0.0 s
After income merge: (90910, 11) | merge took 0.0 s
Head (income):
   idpers  year  income_imputed
0    5101  2019        117000.0
1    5103  2019             NaN
2    5104  2019             NaN
3    5201  2019             NaN
4   13101  2019             NaN
5   13102  2019         40400.0
6   13103  2019             NaN
7   13104  2019             NaN
8   21101  2019         27600.0
9   27101  2019          8500.0
TOTAL income block took 2.1 s


## 5) Build minimal analysis dataset and export

In [16]:
# household structure best-effort
nbpers_col = pick_by_regex(df_panel, [r"^nbpers\d{2}$", r"^nbpers$"])
nbkid_col  = pick_by_regex(df_panel, [r"^nbkid\d{2}$", r"^nbkid$"])

df_panel["nbpers"] = safe_to_numeric(df_panel[nbpers_col]) if nbpers_col else np.nan
df_panel["nbkid"]  = safe_to_numeric(df_panel[nbkid_col]) if nbkid_col else np.nan

vars_keep = [
    "idpers","idhous","year","wave",
    "age","sex","edyear","isced",
    "sport_raw","health_raw","income_imputed",
    "nbpers","nbkid",
]

df_out = df_panel[vars_keep].copy()
print("Export dataset shape:", df_out.shape)
display(df_out.head())

OUTPUT_PATH = "analysis_dataset_step1_panel_2019_2023.csv"
df_out.to_csv(OUTPUT_PATH, index=False)
print("✅ Saved:", OUTPUT_PATH)


Export dataset shape: (90910, 13)


,idpers,idhous,year,wave,age,sex,edyear,isced,sport_raw,health_raw,income_imputed,nbpers,nbkid
0,5101,51,2019,21,58.0,1.0,19.0,51.0,2.0,2.0,117000.0,NaN,NaN
1,5103,52,2019,21,27.0,1.0,9.0,20.0,NaN,NaN,NaN,NaN,NaN
2,5104,51,2019,21,58.0,2.0,12.0,32.0,NaN,NaN,NaN,NaN,NaN
3,5201,52,2019,21,25.0,2.0,12.0,32.0,NaN,NaN,NaN,NaN,NaN
4,13101,131,2019,21,47.0,1.0,12.0,32.0,NaN,NaN,NaN,NaN,NaN


✅ Saved: analysis_dataset_step1_panel_2019_2023.csv


In [17]:
import re

cols = df_long.columns

def show(pattern, n=20):
    hits = [c for c in cols if re.search(pattern, c)]
    print(pattern, "->", hits[:n])

# Demografie
show(r"^age")
show(r"^sex")
show(r"^edyear")
show(r"^isced")

# Sport & Gesundheit nach Jahr-Suffix (19–23)
show(r"^p19a")
show(r"^p19c")
show(r"^p20a")
show(r"^p20c")
show(r"^p21a")
show(r"^p21c")
show(r"^p22a")
show(r"^p22c")
show(r"^p23a")
show(r"^p23c")


^age -> ['age19', 'age20', 'age21', 'age22', 'age23']
^sex -> ['sex19', 'sex20', 'sex21', 'sex22', 'sex23']
^edyear -> ['edyear19', 'edyear20', 'edyear21', 'edyear22', 'edyear23']
^isced -> ['isced19', 'isced20', 'isced21', 'isced22', 'isced23']
^p19a -> ['p19a01', 'p19a04', 'p19a05', 'p19a06', 'p19a14', 'p19a08', 'p19a10', 'p19a11', 'p19a13', 'p19a17', 'p19a19', 'p19a09', 'p19a09a', 'p19a09b', 'p19a09c', 'p19a09d', 'p19a120', 'p19a15', 'p19a18a', 'p19a18b']
^p19c -> ['p19c44', 'p19c47', 'p19c48', 'p19c49', 'p19c50', 'p19c01', 'p19c02', 'p19c03', 'p19c04a', 'p19c05a', 'p19c06a', 'p19c07a', 'p19c08', 'p19c19a', 'p19c11', 'p19c12', 'p19c15', 'p19c186', 'p19c191', 'p19c193']
^p20a -> ['p20a01', 'p20a04', 'p20a05', 'p20a06']
^p20c -> ['p20c44', 'p20c47', 'p20c48', 'p20c49', 'p20c50', 'p20c01', 'p20c02', 'p20c03', 'p20c04a', 'p20c05a', 'p20c06a', 'p20c07a', 'p20c08', 'p20c19a', 'p20c11', 'p20c12', 'p20c15', 'p20c186', 'p20c191', 'p20c193']
^p21a -> ['p21a01', 'p21a04', 'p21a05', 'p21a06']
^